In [1]:
import pandas as pd
import lightgbm as lgb
import os
import gc
import pickle
from sklearn import preprocessing, model_selection

In [2]:
#"""
def get_type_feature_all_(sample, train_df, key, on, type_c, mark):
    if type_c == "count":
        tmp = pd.DataFrame(train_df[key + [on]].groupby(key)[on].count()).reset_index()
    if type_c == "mean":
        tmp = pd.DataFrame(train_df[key + [on]].groupby(key)[on].mean()).reset_index()
    if type_c == "nunique":
        tmp = pd.DataFrame(train_df[key + [on]].groupby(key)[on].nunique()).reset_index()
    if type_c == "max":
        tmp = pd.DataFrame(train_df[key + [on]].groupby(key)[on].max()).reset_index()
    if type_c == "min":
        tmp = pd.DataFrame(train_df[key + [on]].groupby(key)[on].min()).reset_index()
    if type_c == "sum":
        tmp = pd.DataFrame(train_df[key + [on]].groupby(key)[on].sum()).reset_index()
    if type_c == "std":
        tmp = pd.DataFrame(train_df[key + [on]].groupby(key)[on].std()).reset_index()
    if type_c == "median":
        tmp = pd.DataFrame(train_df[key + [on]].groupby(key)[on].median()).reset_index()
    tmp.columns = key + [mark + "_" + "_".join(key) + '_%s_' % type_c + on]
    tmp[mark + "_" + "_".join(key) + '_%s_' % type_c + on] = tmp[
        mark + "_" + "_".join(key) + '_%s_' % type_c + on].astype('float32')
    sample = sample.merge(tmp, on=key, how='left')
    del tmp, train_df
    gc.collect()
    return sample, mark + "_" + "_".join(key) + '_%s_' % type_c + on
#"""
def get_type_feature_all(sample,train_df,key,on,type_c,mark):
    pickle_path="../pickle/"
    filename = "_".join([mark+"_%s_features"%type_c, "_".join(key), on, str(len(sample))]) + ".pkl"
    try:
        with open(pickle_path + filename, "rb") as fp:
            print("load {} {} feature from pickle file: key: {}, on: {}...".format(mark,type_c,"_".join(key), on))
            col = pickle.load(fp)
        for c in col.columns:
            sample[c] = col[c]
        gc.collect()
    except:
        print('get {} {} feature, key: {}, on: {}'.format(mark,type_c,"_".join(key), on))
        if type_c=="count":
            tmp = pd.DataFrame(train_df[key+[on]].groupby(key)[on].count()).reset_index()
        if type_c=="mean":
            tmp = pd.DataFrame(train_df[key+[on]].groupby(key)[on].mean()).reset_index()
        if type_c=="nunique":
            tmp = pd.DataFrame(train_df[key+[on]].groupby(key)[on].nunique()).reset_index()
        if type_c=="max":
            tmp = pd.DataFrame(train_df[key+[on]].groupby(key)[on].max()).reset_index()
        if type_c=="min":
            tmp = pd.DataFrame(train_df[key+[on]].groupby(key)[on].min()).reset_index()
        if type_c=="sum":
            tmp = pd.DataFrame(train_df[key+[on]].groupby(key)[on].sum()).reset_index()
        if type_c=="std":
            tmp = pd.DataFrame(train_df[key+[on]].groupby(key)[on].std()).reset_index()
        if type_c=="median":
            tmp = pd.DataFrame(train_df[key+[on]].groupby(key)[on].median()).reset_index()
        tmp.columns = key+[mark+"_"+"_".join(key) + '_%s_'%type_c + on]
        tmp[mark+"_"+"_".join(key) + '_%s_'%type_c + on] = tmp[mark+"_"+"_".join(key) + '_%s_'%type_c + on].astype('float32')
        sample = sample.merge(tmp, on=key, how='left')
        with open(pickle_path + filename, "wb") as fp:
            col = sample[[mark+"_"+"_".join(key) + '_%s_'%type_c + on]].copy()
            pickle.dump(col, fp)
        del tmp
    #if save_only:
        #for i in col.columns:
            #del sample[i]
    del col,train_df
    gc.collect()
    return sample, mark + "_" + "_".join(key) + '_%s_' % type_c + on


In [3]:
train_data=pd.read_csv("../data/train.csv")
test_data=pd.read_csv("../data/test.csv")
sub=pd.read_csv("../data/sample_submission.csv")

In [4]:
train_data.columns

Index(['article_id', 'date', 'baike_id_1h', 'price', 'price_diff', 'author',
       'level1', 'level2', 'level3', 'level4', 'brand', 'mall', 'url',
       'comments_1h', 'zhi_1h', 'buzhi_1h', 'favorite_1h', 'orders_1h',
       'baike_id_2h', 'comments_2h', 'zhi_2h', 'buzhi_2h', 'favorite_2h',
       'orders_2h', 'orders_3h_15h'],
      dtype='object')

In [5]:
def create_features(data):
    data["order_1h_2h"]=data["orders_1h"]+data["orders_2h"]
    
    date_max=data.groupby(["date"]).article_id.max().reset_index()
    date_max.columns=["date","date_max"]
    data_min=date_max.copy()
    data_min.columns=["date","date_min"]
    data_min["date"]=data_min["date"]+1
    
    data=data.merge(date_max,on="date",how="left")
    data=data.merge(data_min,on="date",how="left")
    data["date_min"]=data["date_min"].fillna(0)
    
    data["article_id"]=(data["article_id"]-data["date_min"])/(data["date_max"]-data["date_min"])
    
    data['hour']=data["article_id"].apply(lambda x:int(x*24))
    
    data["dayofweek"]=data["date"]%7
    
    del data["date_min"],data["date_max"]
    
    
    cat_features=[['baike_id_1h'],['author'],['level1'], ['level2'], ['level3'], ['level4'], ['brand'], ['mall'], ['url'],['dayofweek'],
                  ['author','level1'],['author','level2'],['author','level3'],['author','level4'],['author','brand'],['author','mall'],
                  ['mall','brand'],['mall','level1'],['mall','level2'],['mall','level3'],['mall','level4'],
                  ['brand','level1'],['brand','level2'],['brand','level3'],['brand','level4'],
                  ['baike_id_1h','mall'],
                  ["mall","brand","level1"],["mall","brand","level2"],["mall","brand","level3"],["mall","brand","level4"]

                 ]
    num_features=['comments_1h','comments_2h',
                  'zhi_1h','zhi_2h',
                  'buzhi_1h','buzhi_2h',
                  'favorite_1h','favorite_2h',
                  'orders_1h','orders_2h','order_1h_2h','article_id']
    
    for cat in cat_features:
        data["%s_%s_-1"%("_".join(cat),"article_id")]=data.groupby(cat)["article_id"].shift(-1)-data["article_id"]
        data["%s_%s_1"%("_".join(cat),"article_id")]=data.groupby(cat)["article_id"].shift(1)-data["article_id"]

        data,_=get_type_feature_all(data, data, cat,"%s_%s_-1"%("_".join(cat),"article_id") , "mean", "fe")
        data,_=get_type_feature_all(data, data, cat,"%s_%s_1"%("_".join(cat),"article_id") , "mean", "fe")
    
    for cat in cat_features:
        data,_=get_type_feature_all(data, data, cat+["hour"],"article_id" , "mean", "fe")
        data,_=get_type_feature_all(data, data, cat,"article_id" , "count", "fe")
        
        data,_=get_type_feature_all(data, data, cat,"price" , "mean", "fe")
        data,_=get_type_feature_all(data, data, cat,"price" , "max", "fe")
        data,_=get_type_feature_all(data, data, cat,"price" , "min", "fe")
        

        for num in num_features:
            data,_=get_type_feature_all(data, data, cat,num , "mean", "fe")
            data,_=get_type_feature_all(data, data, cat,num , "sum", "fe")

    
    for cat in cat_features:
        data["%s_%s_-1"%("_".join(cat),"article_id")]=data.groupby(cat)["article_id"].shift(-1)-data["article_id"]
        data["%s_%s_1"%("_".join(cat),"article_id")]=data.groupby(cat)["article_id"].shift(1)-data["article_id"]

        data["%s_%s_-2"%("_".join(cat),"article_id")]=data.groupby(cat)["article_id"].shift(-2)-data["article_id"]
        data["%s_%s_2"%("_".join(cat),"article_id")]=data.groupby(cat)["article_id"].shift(2)-data["article_id"]
        
        data["%s_%s_-3"%("_".join(cat),"article_id")]=data.groupby(cat)["article_id"].shift(-3)-data["article_id"]
        data["%s_%s_-4"%("_".join(cat),"article_id")]=data.groupby(cat)["article_id"].shift(-4)-data["article_id"]
        data["%s_%s_-5"%("_".join(cat),"article_id")]=data.groupby(cat)["article_id"].shift(-5)-data["article_id"]

        data["%s_%s_3"%("_".join(cat),"article_id")]=data.groupby(cat)["article_id"].shift(3)-data["article_id"]
        data["%s_%s_4"%("_".join(cat),"article_id")]=data.groupby(cat)["article_id"].shift(4)-data["article_id"]
        #data,_=get_type_feature_all(data, data, cat+["orders_2h"],"article_id" , "count", "fe")
        #"""
        for num in ['orders_1h','orders_2h','order_1h_2h']:
            data["%s_%s_-1"%("_".join(cat),num)]=data.groupby(cat)[num].shift(-1)
            data["%s_%s_1"%("_".join(cat),num)]=data.groupby(cat)[num].shift(1)
        #data["%s_%s_-1"%("_".join(cat),'price')]=data.groupby(cat)['price'].shift(-1)-data['price']
            #data["%s_%s_date_-1"%("_".join(cat),num)]=data.groupby(cat+["date"])[num].shift(-1)
    
    return data

In [6]:
data=pd.concat([train_data,
                #test_data,
               ]).reset_index(drop=True)

data["orders_1h_2h"]=data["orders_1h"]+data["orders_2h"]
#"""
features_data=data[data.date<110].copy()
def get_data(dt):
    data_tmp=data[data.date==dt].copy()
    features=features_data[features_data.date!=dt].copy()
    features["r1"]=features["orders_1h"]/features["orders_3h_15h"]
    features["r2"]=features["orders_2h"]/features["orders_3h_15h"]
    features["r3"]=features["orders_1h_2h"]/features["orders_3h_15h"]
    for cat in ['baike_id_1h','author','level1', 'level2', 'level3', 'level4', 'brand', 'mall', 'url']:
        data_tmp,_=get_type_feature_all_(data_tmp, features, [cat],"orders_3h_15h" , "mean", "features")
        data_tmp,_=get_type_feature_all_(data_tmp, features, [cat],"r1" , "mean", "features")
        data_tmp,_=get_type_feature_all_(data_tmp, features, [cat],"r2" , "mean", "features")
        data_tmp,_=get_type_feature_all_(data_tmp, features, [cat],"r3" , "mean", "features")
    return data_tmp



data_all=[]
for dt in range(1,124):
    data_tmp=get_data(dt)
    data_all.append(data_tmp)
data_all=pd.concat(data_all).reset_index(drop=True)
#"""
data=create_features(data_all)

load fe mean feature from pickle file: key: baike_id_1h, on: baike_id_1h_article_id_-1...
load fe mean feature from pickle file: key: baike_id_1h, on: baike_id_1h_article_id_1...
load fe mean feature from pickle file: key: author, on: author_article_id_-1...
load fe mean feature from pickle file: key: author, on: author_article_id_1...
load fe mean feature from pickle file: key: level1, on: level1_article_id_-1...
load fe mean feature from pickle file: key: level1, on: level1_article_id_1...
load fe mean feature from pickle file: key: level2, on: level2_article_id_-1...
load fe mean feature from pickle file: key: level2, on: level2_article_id_1...
load fe mean feature from pickle file: key: level3, on: level3_article_id_-1...
load fe mean feature from pickle file: key: level3, on: level3_article_id_1...
load fe mean feature from pickle file: key: level4, on: level4_article_id_-1...
load fe mean feature from pickle file: key: level4, on: level4_article_id_1...
load fe mean feature from 

load fe mean feature from pickle file: key: author, on: zhi_2h...
load fe sum feature from pickle file: key: author, on: zhi_2h...
load fe mean feature from pickle file: key: author, on: buzhi_1h...
load fe sum feature from pickle file: key: author, on: buzhi_1h...
load fe mean feature from pickle file: key: author, on: buzhi_2h...
load fe sum feature from pickle file: key: author, on: buzhi_2h...
load fe mean feature from pickle file: key: author, on: favorite_1h...
load fe sum feature from pickle file: key: author, on: favorite_1h...
load fe mean feature from pickle file: key: author, on: favorite_2h...
load fe sum feature from pickle file: key: author, on: favorite_2h...
load fe mean feature from pickle file: key: author, on: orders_1h...
load fe sum feature from pickle file: key: author, on: orders_1h...
load fe mean feature from pickle file: key: author, on: orders_2h...
load fe sum feature from pickle file: key: author, on: orders_2h...
load fe mean feature from pickle file: key:

load fe mean feature from pickle file: key: level4, on: buzhi_2h...
load fe sum feature from pickle file: key: level4, on: buzhi_2h...
load fe mean feature from pickle file: key: level4, on: favorite_1h...
load fe sum feature from pickle file: key: level4, on: favorite_1h...
load fe mean feature from pickle file: key: level4, on: favorite_2h...
load fe sum feature from pickle file: key: level4, on: favorite_2h...
load fe mean feature from pickle file: key: level4, on: orders_1h...
load fe sum feature from pickle file: key: level4, on: orders_1h...
load fe mean feature from pickle file: key: level4, on: orders_2h...
load fe sum feature from pickle file: key: level4, on: orders_2h...
load fe mean feature from pickle file: key: level4, on: order_1h_2h...
load fe sum feature from pickle file: key: level4, on: order_1h_2h...
load fe mean feature from pickle file: key: level4, on: article_id...
load fe sum feature from pickle file: key: level4, on: article_id...
load fe mean feature from pic

load fe sum feature from pickle file: key: dayofweek, on: orders_1h...
load fe mean feature from pickle file: key: dayofweek, on: orders_2h...
load fe sum feature from pickle file: key: dayofweek, on: orders_2h...
load fe mean feature from pickle file: key: dayofweek, on: order_1h_2h...
load fe sum feature from pickle file: key: dayofweek, on: order_1h_2h...
load fe mean feature from pickle file: key: dayofweek, on: article_id...
load fe sum feature from pickle file: key: dayofweek, on: article_id...
load fe mean feature from pickle file: key: author_level1_hour, on: article_id...
load fe count feature from pickle file: key: author_level1, on: article_id...
load fe mean feature from pickle file: key: author_level1, on: price...
load fe max feature from pickle file: key: author_level1, on: price...
load fe min feature from pickle file: key: author_level1, on: price...
load fe mean feature from pickle file: key: author_level1, on: comments_1h...
load fe sum feature from pickle file: key:

load fe sum feature from pickle file: key: author_level4, on: buzhi_2h...
load fe mean feature from pickle file: key: author_level4, on: favorite_1h...
load fe sum feature from pickle file: key: author_level4, on: favorite_1h...
load fe mean feature from pickle file: key: author_level4, on: favorite_2h...
load fe sum feature from pickle file: key: author_level4, on: favorite_2h...
load fe mean feature from pickle file: key: author_level4, on: orders_1h...
load fe sum feature from pickle file: key: author_level4, on: orders_1h...
load fe mean feature from pickle file: key: author_level4, on: orders_2h...
load fe sum feature from pickle file: key: author_level4, on: orders_2h...
load fe mean feature from pickle file: key: author_level4, on: order_1h_2h...
load fe sum feature from pickle file: key: author_level4, on: order_1h_2h...
load fe mean feature from pickle file: key: author_level4, on: article_id...
load fe sum feature from pickle file: key: author_level4, on: article_id...
load f

load fe sum feature from pickle file: key: mall_level1, on: zhi_2h...
load fe mean feature from pickle file: key: mall_level1, on: buzhi_1h...
load fe sum feature from pickle file: key: mall_level1, on: buzhi_1h...
load fe mean feature from pickle file: key: mall_level1, on: buzhi_2h...
load fe sum feature from pickle file: key: mall_level1, on: buzhi_2h...
load fe mean feature from pickle file: key: mall_level1, on: favorite_1h...
load fe sum feature from pickle file: key: mall_level1, on: favorite_1h...
load fe mean feature from pickle file: key: mall_level1, on: favorite_2h...
load fe sum feature from pickle file: key: mall_level1, on: favorite_2h...
load fe mean feature from pickle file: key: mall_level1, on: orders_1h...
load fe sum feature from pickle file: key: mall_level1, on: orders_1h...
load fe mean feature from pickle file: key: mall_level1, on: orders_2h...
load fe sum feature from pickle file: key: mall_level1, on: orders_2h...
load fe mean feature from pickle file: key: 

load fe sum feature from pickle file: key: brand_level1, on: comments_2h...
load fe mean feature from pickle file: key: brand_level1, on: zhi_1h...
load fe sum feature from pickle file: key: brand_level1, on: zhi_1h...
load fe mean feature from pickle file: key: brand_level1, on: zhi_2h...
load fe sum feature from pickle file: key: brand_level1, on: zhi_2h...
load fe mean feature from pickle file: key: brand_level1, on: buzhi_1h...
load fe sum feature from pickle file: key: brand_level1, on: buzhi_1h...
load fe mean feature from pickle file: key: brand_level1, on: buzhi_2h...
load fe sum feature from pickle file: key: brand_level1, on: buzhi_2h...
load fe mean feature from pickle file: key: brand_level1, on: favorite_1h...
load fe sum feature from pickle file: key: brand_level1, on: favorite_1h...
load fe mean feature from pickle file: key: brand_level1, on: favorite_2h...
load fe sum feature from pickle file: key: brand_level1, on: favorite_2h...
load fe mean feature from pickle file:

load fe mean feature from pickle file: key: baike_id_1h_mall, on: price...
load fe max feature from pickle file: key: baike_id_1h_mall, on: price...
load fe min feature from pickle file: key: baike_id_1h_mall, on: price...
load fe mean feature from pickle file: key: baike_id_1h_mall, on: comments_1h...
load fe sum feature from pickle file: key: baike_id_1h_mall, on: comments_1h...
load fe mean feature from pickle file: key: baike_id_1h_mall, on: comments_2h...
load fe sum feature from pickle file: key: baike_id_1h_mall, on: comments_2h...
load fe mean feature from pickle file: key: baike_id_1h_mall, on: zhi_1h...
load fe sum feature from pickle file: key: baike_id_1h_mall, on: zhi_1h...
load fe mean feature from pickle file: key: baike_id_1h_mall, on: zhi_2h...
load fe sum feature from pickle file: key: baike_id_1h_mall, on: zhi_2h...
load fe mean feature from pickle file: key: baike_id_1h_mall, on: buzhi_1h...
load fe sum feature from pickle file: key: baike_id_1h_mall, on: buzhi_1h..

load fe mean feature from pickle file: key: mall_brand_level3, on: favorite_2h...
load fe sum feature from pickle file: key: mall_brand_level3, on: favorite_2h...
load fe mean feature from pickle file: key: mall_brand_level3, on: orders_1h...
load fe sum feature from pickle file: key: mall_brand_level3, on: orders_1h...
load fe mean feature from pickle file: key: mall_brand_level3, on: orders_2h...
load fe sum feature from pickle file: key: mall_brand_level3, on: orders_2h...
load fe mean feature from pickle file: key: mall_brand_level3, on: order_1h_2h...
load fe sum feature from pickle file: key: mall_brand_level3, on: order_1h_2h...
load fe mean feature from pickle file: key: mall_brand_level3, on: article_id...
load fe sum feature from pickle file: key: mall_brand_level3, on: article_id...
load fe mean feature from pickle file: key: mall_brand_level4_hour, on: article_id...
load fe count feature from pickle file: key: mall_brand_level4, on: article_id...
load fe mean feature from p

In [7]:
print(data.shape)

(1807657, 1445)


In [8]:
train=data[(data.date<110)].copy()
#train=data[data.date<117].copy()
#train=data[(data.date>=103)&(data.date<110)].copy()
valid=data[(data.date>=110)&(data.date<117)].copy()

In [9]:
features = list(train[:1].drop(['date', 'orders_3h_15h'],axis=1).columns)

In [13]:
train_set = lgb.Dataset(train[features], label=train["orders_3h_15h"])
val_set = lgb.Dataset(valid[features], label=valid["orders_3h_15h"])

params_lgbm = {
          'task': 'train',
          'boosting_type': 'gbdt',
          'objective': 'multiclass',
          'metric': 'multi_logloss',
          'min_child_weight': 1.5,
          'num_leaves': 2**5,
          'max_depth': 5,
          'lambda_l2': 0.7,
          'subsample': 0.7,
          'colsample_bytree': 0.7,
          'colsample_bylevel': 0.7,
          'learning_rate': 0.1,
          'num_class': 10,
          #'tree_method': 'exact',
          'seed': 2021,
          'nthread': 18,
          'verbose': -1,
          }


model = lgb.train(params_lgbm, 
                  train_set, 
                  1000, 
                  #valid_sets=val_set, 
                  #feval=feval_RMSPE,
                  #verbose_eval= 500,
                  #early_stopping_rounds=500,
                  #categorical_feature=cats,
                 )

In [14]:
valid_score=train_data[(train_data.date>=110)].copy()[['article_id', 'date','orders_1h','orders_2h','orders_3h_15h']].copy()
pred=model.predict(valid[features])
valid_pred=pd.DataFrame(pred)
valid_pred.columns=["multiclass_%s"%i for i in range(10)]
valid_pred["article_id"]=list(valid_score["article_id"])
valid_pred.to_csv("../user_data/train_stacking_multiclass.csv",index=None)

In [25]:
valid_pred["article_id"]=list(valid_score["article_id"])
valid_pred.to_csv("../user_data/train_stacking_multiclass.csv",index=None)

In [26]:
valid_pred

,multiclass_0,multiclass_1,multiclass_2,multiclass_3,multiclass_4,multiclass_5,multiclass_6,multiclass_7,multiclass_8,multiclass_9,article_id
0,0.048265,0.084463,0.066319,0.145173,0.345200,0.223956,8.452200e-02,2.101201e-03,3.559478e-08,2.153364e-08,1664040
1,0.836240,0.130124,0.020551,0.005360,0.006867,0.000844,1.281682e-05,2.101452e-06,6.954286e-10,5.615505e-09,1664289
2,0.012014,0.032319,0.031650,0.028011,0.172983,0.213925,4.898096e-01,1.928680e-02,1.744316e-06,9.463905e-09,1664686
3,0.844435,0.098422,0.036860,0.010262,0.008808,0.001174,3.431173e-05,5.698978e-06,1.176232e-08,3.304335e-09,1666137
4,0.516014,0.257114,0.081919,0.051962,0.086046,0.006804,1.022972e-04,3.837276e-05,1.292047e-08,1.091210e-08,1667990
...,...,...,...,...,...,...,...,...,...,...,...
144956,0.960599,0.035864,0.001359,0.001302,0.000861,0.000011,3.737118e-06,2.489417e-07,1.133266e-09,1.314163e-09,1804751
144957,0.909324,0.068581,0.007901,0.001187,0.012439,0.000445,1.216094e-04,1.330265e-06,1.634624e-09,8.218090e-09,1805971
144958,0.976219,0.022702,0.000523,0.000087,0.000456,0.000010,2.194266e-06,5.312324e-08,2.335591e-11,1.944074e-10,1807148
144959,0.966299,0.029437,0.001713,0.000813,0.001653,0.000082,2.728182e-06,7.329310e-08,2.976977e-12,6.594937e-10,1807424


In [15]:
print("\n".join(("%s: %.2f" % x) for x in
                list(sorted(zip(features, model.feature_importance("gain")), key=lambda x: x[1],
                      
                            reverse=True))[:200]
                ))


orders_1h_2h: 2283380.50
fe_url_mean_order_1h_2h: 1136356.91
order_1h_2h: 890057.66
favorite_2h: 647863.31
orders_2h: 592129.64
fe_url_sum_favorite_2h: 440453.72
features_url_mean_orders_3h_15h: 384116.76
features_author_mean_orders_3h_15h: 197219.15
features_mall_mean_orders_3h_15h: 190922.17
fe_dayofweek_hour_mean_article_id: 159245.14
fe_url_mean_orders_2h: 135397.10
orders_1h: 84797.29
article_id: 79258.90
fe_mall_brand_level3_mean_order_1h_2h: 73527.50
fe_url_sum_comments_2h: 68861.66
features_baike_id_1h_mean_orders_3h_15h: 63588.94
fe_author_brand_mean_order_1h_2h: 63124.72
favorite_1h: 61808.32
price: 61190.13
fe_url_sum_order_1h_2h: 59789.48
fe_mall_mean_price: 59642.43
price_diff: 58445.09
dayofweek_article_id_-4: 58333.77
dayofweek_article_id_-5: 57613.16
fe_mall_brand_level3_mean_orders_2h: 55150.17
fe_mall_brand_level2_mean_order_1h_2h: 49606.32
comments_2h: 49348.32
fe_url_mean_comments_2h: 48529.54
features_brand_mean_orders_3h_15h: 44724.93
fe_mall_hour_mean_article_id:

In [16]:
data=pd.concat([train_data,
                test_data,
               ]).reset_index(drop=True)

data["orders_1h_2h"]=data["orders_1h"]+data["orders_2h"]
#"""
features_data=data[data.date<117].copy()
def get_data(dt):
    data_tmp=data[data.date==dt].copy()
    features=features_data[features_data.date!=dt].copy()
    features["r1"]=features["orders_1h"]/features["orders_3h_15h"]
    features["r2"]=features["orders_2h"]/features["orders_3h_15h"]
    features["r3"]=features["orders_1h_2h"]/features["orders_3h_15h"]
    for cat in ['baike_id_1h','author','level1', 'level2', 'level3', 'level4', 'brand', 'mall', 'url']:
        data_tmp,_=get_type_feature_all_(data_tmp, features, [cat],"orders_3h_15h" , "mean", "features")
        data_tmp,_=get_type_feature_all_(data_tmp, features, [cat],"r1" , "mean", "features")
        data_tmp,_=get_type_feature_all_(data_tmp, features, [cat],"r2" , "mean", "features")
        data_tmp,_=get_type_feature_all_(data_tmp, features, [cat],"r3" , "mean", "features")
    return data_tmp



data_all=[]
for dt in range(1,124):
    data_tmp=get_data(dt)
    data_all.append(data_tmp)
data_all=pd.concat(data_all).reset_index(drop=True)
#"""
data=create_features(data_all)

load fe mean feature from pickle file: key: baike_id_1h, on: baike_id_1h_article_id_-1...
load fe mean feature from pickle file: key: baike_id_1h, on: baike_id_1h_article_id_1...
load fe mean feature from pickle file: key: author, on: author_article_id_-1...
load fe mean feature from pickle file: key: author, on: author_article_id_1...
load fe mean feature from pickle file: key: level1, on: level1_article_id_-1...
load fe mean feature from pickle file: key: level1, on: level1_article_id_1...
load fe mean feature from pickle file: key: level2, on: level2_article_id_-1...
load fe mean feature from pickle file: key: level2, on: level2_article_id_1...
load fe mean feature from pickle file: key: level3, on: level3_article_id_-1...
load fe mean feature from pickle file: key: level3, on: level3_article_id_1...
load fe mean feature from pickle file: key: level4, on: level4_article_id_-1...
load fe mean feature from pickle file: key: level4, on: level4_article_id_1...
load fe mean feature from 

load fe mean feature from pickle file: key: author, on: zhi_2h...
load fe sum feature from pickle file: key: author, on: zhi_2h...
load fe mean feature from pickle file: key: author, on: buzhi_1h...
load fe sum feature from pickle file: key: author, on: buzhi_1h...
load fe mean feature from pickle file: key: author, on: buzhi_2h...
load fe sum feature from pickle file: key: author, on: buzhi_2h...
load fe mean feature from pickle file: key: author, on: favorite_1h...
load fe sum feature from pickle file: key: author, on: favorite_1h...
load fe mean feature from pickle file: key: author, on: favorite_2h...
load fe sum feature from pickle file: key: author, on: favorite_2h...
load fe mean feature from pickle file: key: author, on: orders_1h...
load fe sum feature from pickle file: key: author, on: orders_1h...
load fe mean feature from pickle file: key: author, on: orders_2h...
load fe sum feature from pickle file: key: author, on: orders_2h...
load fe mean feature from pickle file: key:

load fe sum feature from pickle file: key: level4, on: buzhi_2h...
load fe mean feature from pickle file: key: level4, on: favorite_1h...
load fe sum feature from pickle file: key: level4, on: favorite_1h...
load fe mean feature from pickle file: key: level4, on: favorite_2h...
load fe sum feature from pickle file: key: level4, on: favorite_2h...
load fe mean feature from pickle file: key: level4, on: orders_1h...
load fe sum feature from pickle file: key: level4, on: orders_1h...
load fe mean feature from pickle file: key: level4, on: orders_2h...
load fe sum feature from pickle file: key: level4, on: orders_2h...
load fe mean feature from pickle file: key: level4, on: order_1h_2h...
load fe sum feature from pickle file: key: level4, on: order_1h_2h...
load fe mean feature from pickle file: key: level4, on: article_id...
load fe sum feature from pickle file: key: level4, on: article_id...
load fe mean feature from pickle file: key: brand_hour, on: article_id...
load fe count feature f

load fe sum feature from pickle file: key: dayofweek, on: orders_1h...
load fe mean feature from pickle file: key: dayofweek, on: orders_2h...
load fe sum feature from pickle file: key: dayofweek, on: orders_2h...
load fe mean feature from pickle file: key: dayofweek, on: order_1h_2h...
load fe sum feature from pickle file: key: dayofweek, on: order_1h_2h...
load fe mean feature from pickle file: key: dayofweek, on: article_id...
load fe sum feature from pickle file: key: dayofweek, on: article_id...
load fe mean feature from pickle file: key: author_level1_hour, on: article_id...
load fe count feature from pickle file: key: author_level1, on: article_id...
load fe mean feature from pickle file: key: author_level1, on: price...
load fe max feature from pickle file: key: author_level1, on: price...
load fe min feature from pickle file: key: author_level1, on: price...
load fe mean feature from pickle file: key: author_level1, on: comments_1h...
load fe sum feature from pickle file: key:

load fe mean feature from pickle file: key: author_level4, on: buzhi_2h...
load fe sum feature from pickle file: key: author_level4, on: buzhi_2h...
load fe mean feature from pickle file: key: author_level4, on: favorite_1h...
load fe sum feature from pickle file: key: author_level4, on: favorite_1h...
load fe mean feature from pickle file: key: author_level4, on: favorite_2h...
load fe sum feature from pickle file: key: author_level4, on: favorite_2h...
load fe mean feature from pickle file: key: author_level4, on: orders_1h...
load fe sum feature from pickle file: key: author_level4, on: orders_1h...
load fe mean feature from pickle file: key: author_level4, on: orders_2h...
load fe sum feature from pickle file: key: author_level4, on: orders_2h...
load fe mean feature from pickle file: key: author_level4, on: order_1h_2h...
load fe sum feature from pickle file: key: author_level4, on: order_1h_2h...
load fe mean feature from pickle file: key: author_level4, on: article_id...
load fe

load fe mean feature from pickle file: key: mall_level1, on: zhi_2h...
load fe sum feature from pickle file: key: mall_level1, on: zhi_2h...
load fe mean feature from pickle file: key: mall_level1, on: buzhi_1h...
load fe sum feature from pickle file: key: mall_level1, on: buzhi_1h...
load fe mean feature from pickle file: key: mall_level1, on: buzhi_2h...
load fe sum feature from pickle file: key: mall_level1, on: buzhi_2h...
load fe mean feature from pickle file: key: mall_level1, on: favorite_1h...
load fe sum feature from pickle file: key: mall_level1, on: favorite_1h...
load fe mean feature from pickle file: key: mall_level1, on: favorite_2h...
load fe sum feature from pickle file: key: mall_level1, on: favorite_2h...
load fe mean feature from pickle file: key: mall_level1, on: orders_1h...
load fe sum feature from pickle file: key: mall_level1, on: orders_1h...
load fe mean feature from pickle file: key: mall_level1, on: orders_2h...
load fe sum feature from pickle file: key: mal

load fe sum feature from pickle file: key: brand_level1, on: comments_2h...
load fe mean feature from pickle file: key: brand_level1, on: zhi_1h...
load fe sum feature from pickle file: key: brand_level1, on: zhi_1h...
load fe mean feature from pickle file: key: brand_level1, on: zhi_2h...
load fe sum feature from pickle file: key: brand_level1, on: zhi_2h...
load fe mean feature from pickle file: key: brand_level1, on: buzhi_1h...
load fe sum feature from pickle file: key: brand_level1, on: buzhi_1h...
load fe mean feature from pickle file: key: brand_level1, on: buzhi_2h...
load fe sum feature from pickle file: key: brand_level1, on: buzhi_2h...
load fe mean feature from pickle file: key: brand_level1, on: favorite_1h...
load fe sum feature from pickle file: key: brand_level1, on: favorite_1h...
load fe mean feature from pickle file: key: brand_level1, on: favorite_2h...
load fe sum feature from pickle file: key: brand_level1, on: favorite_2h...
load fe mean feature from pickle file:

load fe mean feature from pickle file: key: baike_id_1h_mall, on: price...
load fe max feature from pickle file: key: baike_id_1h_mall, on: price...
load fe min feature from pickle file: key: baike_id_1h_mall, on: price...
load fe mean feature from pickle file: key: baike_id_1h_mall, on: comments_1h...
load fe sum feature from pickle file: key: baike_id_1h_mall, on: comments_1h...
load fe mean feature from pickle file: key: baike_id_1h_mall, on: comments_2h...
load fe sum feature from pickle file: key: baike_id_1h_mall, on: comments_2h...
load fe mean feature from pickle file: key: baike_id_1h_mall, on: zhi_1h...
load fe sum feature from pickle file: key: baike_id_1h_mall, on: zhi_1h...
load fe mean feature from pickle file: key: baike_id_1h_mall, on: zhi_2h...
load fe sum feature from pickle file: key: baike_id_1h_mall, on: zhi_2h...
load fe mean feature from pickle file: key: baike_id_1h_mall, on: buzhi_1h...
load fe sum feature from pickle file: key: baike_id_1h_mall, on: buzhi_1h..

load fe sum feature from pickle file: key: mall_brand_level3, on: favorite_2h...
load fe mean feature from pickle file: key: mall_brand_level3, on: orders_1h...
load fe sum feature from pickle file: key: mall_brand_level3, on: orders_1h...
load fe mean feature from pickle file: key: mall_brand_level3, on: orders_2h...
load fe sum feature from pickle file: key: mall_brand_level3, on: orders_2h...
load fe mean feature from pickle file: key: mall_brand_level3, on: order_1h_2h...
load fe sum feature from pickle file: key: mall_brand_level3, on: order_1h_2h...
load fe mean feature from pickle file: key: mall_brand_level3, on: article_id...
load fe sum feature from pickle file: key: mall_brand_level3, on: article_id...
load fe mean feature from pickle file: key: mall_brand_level4_hour, on: article_id...
load fe count feature from pickle file: key: mall_brand_level4, on: article_id...
load fe mean feature from pickle file: key: mall_brand_level4, on: price...
load fe max feature from pickle f

In [17]:
train=data[data.date<117].copy()
valid=data[(data.date>=110)&(data.date<117)].copy()
test=data[data.date>=117].copy()

In [18]:
train_set = lgb.Dataset(train[features], label=train["orders_3h_15h"])
val_set = lgb.Dataset(valid[features], label=valid["orders_3h_15h"])

model = lgb.train(params_lgbm, 
                  train_set, 
                  1000, 
                  #valid_sets=val_set, 
                  #feval=feval_RMSPE,
                  #verbose_eval= 500,
                  #early_stopping_rounds=500,
                 )

[LightGBM] [Warning] Unknown parameter: colsample_bylevel


In [19]:
pred=model.predict(test[features])
test_pred=pd.DataFrame(pred)
test_pred.columns=["multiclass_%s"%i for i in range(10)]
test_pred["article_id"]=test_data["article_id"]
test_pred.to_csv("../user_data/test_stacking_multiclass.csv",index=None)

In [21]:
test_pred.shape

(149589, 11)